<a href="https://colab.research.google.com/github/matteoalberti/Lectures_AdvCV_Experis2020/blob/main/day4/DL4CV_4_ObjectDet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Object Detection

      This is an extra lecture on Obj Det. Material must be re-write for sharing with ad hoc slides and text. 

      Only for AdvanceCV models Lecture : Experis Academy 2020





<img src="https://cdn.analyticsvidhya.com/wp-content/uploads/2019/08/real_time_object_detection.jpg" width="500">



   Email: m.alberti@deeplearningitalia.com

   Linkedin:
   [linkedin_matteo_alberti](www.linkedin.com/in/matteo-alberti-695570110)

# One step at a time : 

      Going Backwards!



## Which will be the core family behind deep learning models?

      We are working still with images

## How should we evaluate?

*Evaluation Metrics: mAP*

**mAP**, short for “mean average precision”.
- Range : from 0 to 100
- Higher value is better.

*What is?*

Combine all detections from all test images to draw a precision-recall curve (PR curve) for each class; The “average precision” (AP) is the area under the PR curve. (**Do you remember something?**)

Given that target objects are in different classes, we first compute AP separately for each class, and then average over classes.
A detection is a true positive if it has “intersection over union” (IoU) with a ground-truth box greater than some threshold (usually 0.5; if so, the metric is “mAP@0.5”)

### If you think carefully we have quite all blocks needed!

      Convolutions + new criterion

      What else?
      - New Data Structure
      - New Models

## Introduction CNN for Obj Detection

- <font color=BE3315>**R-CNN idea** </font> 
- <font color=C24024>**Variants like Fast, Faster and Mask** </font> 
- <font color=E15234>**SSD** </font> 

- <font color=E35F2A>**YOLO** </font>

## R-CNN

R-CNN or “Region-based Convolutional Neural Networks”. 

Two steps :
- using selective search and it identifies a manageable number of bounding-box object region candidates (“region of interest” or “**RoI**”).
- And then it extracts CNN features from each region independently for classification.

![](https://lilianweng.github.io/lil-log/assets/images/RCNN.png)

## WORKFLOW

1) Pre-train a CNN network on image classification tasks. Like **VGG or ResNet**. *Pretrained models are welcome!* . The classification task involves N classes

2) Propose category-independent regions of interest by **selective search**





-----------------------------------------------------------------

*What is selective search??*

<img src="https://lilianweng.github.io/lil-log/assets/images/selective-search-algorithm.png" width="400">

*Main idea*

- First the similarities between all neighbouring regions are calculated.
- The two most similar regions are grouped together, and new similarities are calculated between the resulting region and its neighbours.

-----------------------------------------------------------------

3) Region candidates are warped to have a fixed size as required by CNN. (**Why?**)

4) Fine tuning on warped regions with number of classes + 1 (background)

5) Given every image region, one forward propagation through the CNN generates a feature vector. This feature vector is then consumed by a **binary** SVM (can you say me why SVM??) trained for each class **independently**. 

- Proposed regions are the ones with IoU (intersection over union) higher than a given threshold

6) To reduce the localization errors, a regression model is trained to correct the predicted detection window on bounding box correction offset using CNN features 







*We will not define into details this step. If you're interested this is called Bounding Box Regression* **BUT** the main idea is the following :

<img src="https://media.springernature.com/original/springer-static/image/chp%3A10.1007%2F978-3-030-20876-9_24/MediaObjects/484523_1_En_24_Fig1_HTML.png" width="500">

## Fast R-CNN

<img src="https://lilianweng.github.io/lil-log/assets/images/fast-RCNN.png" width="500">



*Computation Sharing for speed-up R-CNN*

- Instead of extracting CNN feature vectors independently for each region proposal, this model aggregates them into one CNN forward pass over the entire image (This is obvious if you think about the logic behind MLP to CNN) and the region proposals share this feature matrix. 

- The same feature matrix is branched out to be used for learning the object classifier and the bounding-box regressor



## WORKFLOW

1-2 like previous one

3) Modify the CNN architecture :
- Replace the last max pooling layer of the pre-trained CNN with a **RoI pooling layer**. The RoI pooling layer outputs fixed-length feature vectors of region proposals. Sharing the CNN computation makes a lot of sense, as many region proposals of the same images are highly overlapped.

- Replace the last fully connected layer and the last softmax layer (K classes) with a fully connected layer and softmax over K + 1 classes. (*as before*)

    - We have a different Loss Function that will optimize classification + localization



----------------------------------------------------
*What is a ROI pooling?*

It is a type of max pooling to convert features in the projected region of the image of any size, h x w, into a small fixed window, H x W. The input region is divided into H x W grids, approximately every subwindow of size h/H x w/W. Then apply max-pooling in each grid.

----------------------------------------------------
$$$$

4-5) like previous model : softmax on K+1 classes and BB regression


**Anyhow**

Fast R-CNN is much faster in both training and testing time. However, the improvement is not dramatic because the region proposals are generated separately by another model and that is very expensive



## Faster R-CNN

<img src="https://lilianweng.github.io/lil-log/assets/images/faster-RCNN.png" width="500">




*As said before :*

-  integrate the region proposal algorithm into the CNN model.

## WORKFLOW

1) equal to previous models
2) Fine-tune the RPN (region proposal network) end-to-end for the region proposal task, which is initialized by the pre-train image classifier. Positive samples have IoU higher of a give threshold (as before)
  - Slide a small n x n spatial window over the conv feature map of the entire image.
  - At the center of each sliding window, we predict multiple regions of various scales and ratios simultaneously

3) Train a Fast R-CNN object detection model using the proposals generated by the current RPN

4) Then use the Fast R-CNN network to initialize RPN training. While keeping the shared convolutional layers, only fine-tune the RPN-specific layers. At this stage, RPN and the detection network have shared convolutional layers

5) Fine-tune the unique layers of Fast R-CNN

*Step 4-5 can be repeated to train RPN and Fast R-CNN alternatively if needed.*

## Mask R-CNN


- Not a faster model
- First attempt of CNN to Image Segmentation
  - Extend Faster R-CNN to **pixel-level**

<img src="https://lilianweng.github.io/lil-log/assets/images/mask-rcnn.png" width="500">


Because pixel-level segmentation requires much more fine-grained alignment than bounding boxes, mask R-CNN improves the **RoI pooling layer**


## R-CNN Family :

![](https://lilianweng.github.io/lil-log/assets/images/rcnn-family-summary.png)

## From two-stage to one-stage detector

*Until now we had two different stages :*

      - Define RoI
      - classifier of RoI

*Let's consider another approach*

*Why?* 

Consider that we are interested in a Real-Time object detector. Our inference must be super fast!!

## YOLO 
      [You Only Look Once]

<img src="https://lilianweng.github.io/lil-log/assets/images/yolo-network-architecture.png" width="500">




### WORKFLOW

1) Pre-train a CNN network on image classification task.

2) Split an image into S×S cells. If an object’s center falls into a cell, that cell is “responsible” for detecting the existence of that object. Each cell predicts : 
- the location of B bounding boxes 

      (4d tuple)
      --->  (center x-coord, center y-coord, width, height)

- a confidence score

      P(containing an object) x IoU(pred, truth)


-  a probability of object class conditioned on the existence of an object in the bounding box.

        If box has confidence score :
        ---> array of probability for each class

## SSD

      Single Shot MultiBox Detector

<img src="https://lilianweng.github.io/lil-log/assets/images/SSD-architecture.png" width="500">


Main topics abot SSD :

- Massive use of pre-trained models
  - Pyramidal Features (do you remember the differences between ResNet and VGG?)

- Unlike YOLO, SSD does not split the image into grids of arbitrary size but predicts offset of predefined **anchor boxes**

 